### Kütüphane ve Ayar

In [1]:

import datetime as dt
import pandas as pd
import pymysql
import mysql.connector
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

pd.set_option('display.max_columns', None)

pd.set_option('display.max_columns', None)



#### Veris Seti Oku

In [3]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [4]:
df = df_.copy()

In [5]:
df.shape
df.head()
df.info()
df.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


#### FROM DB

In [6]:
# credentials.
creds = {'user': 'synan',
         'passwd': 'haydegidelum',
         'host': 'db.github.rocks',
         'port': 3306,
         'db': 'group4'}

In [7]:
# MySQL conection string.
connstr = 'mysql+mysqlconnector://{user}:{passwd}@{host}:{port}/{db}'

# sqlalchemy engine for MySQL connection.
conn = create_engine(connstr.format(**creds))


# Data Preperation

In [8]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


def crm_data_prep(dataframe):
    dataframe.dropna(axis=0, inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    return dataframe



In [9]:
check_df(df)

##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                 WHITE METAL LANTERN         6   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom  
##################### NA #####################
Invoice             0

In [10]:
df_prep = crm_data_prep(df)
check_df(df_prep)

##################### Shape #####################
(397925, 9)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1  536365     71053                 WHITE METAL LANTERN       6.0   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER       8.0   

          InvoiceDate  Price  Customer ID         Country  TotalPrice  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom       15.30  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom       20.34  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom     

# Creating RFM Segments

In [11]:

def create_rfm(dataframe):
    today_date = dt.datetime(2011, 12, 11)

    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})

    rfm.columns = ['recency', 'frequency', "monetary"]

    rfm = rfm[(rfm['monetary'] > 0)]


    # RFM SKORLARININ HESAPLANMASI
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])


    # SEGMENTLERIN ISIMLENDIRILMESI
    rfm['rfm_segment'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['rfm_segment'] = rfm['rfm_segment'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "rfm_segment"]]
    return rfm

In [12]:
rfm = create_rfm(df_prep)
rfm.head()

,recency,frequency,monetary,rfm_segment
Customer ID,,,,
12346.0,326,1,310.44,hibernating
12347.0,3,7,4310.00,champions
12348.0,76,4,1770.78,at_risk
12349.0,19,1,1491.72,promising
12350.0,311,1,331.46,hibernating


# Calculated CLTV

In [13]:
def create_cltv_c(dataframe):
    # avg_order_value
    dataframe['avg_order_value'] = dataframe['monetary'] / dataframe['frequency']

    # purchase_frequency
    dataframe["purchase_frequency"] = dataframe['frequency'] / dataframe.shape[0]

    # repeat rate & churn rate
    repeat_rate = dataframe[dataframe.frequency > 1].shape[0] / dataframe.shape[0]
    churn_rate = 1 - repeat_rate

    # profit_margin
    dataframe['profit_margin'] = dataframe['monetary'] * 0.05

    # Customer Value
    dataframe['cv'] = (dataframe['avg_order_value'] * dataframe["purchase_frequency"])

    # Customer Lifetime Value
    dataframe['cltv'] = (dataframe['cv'] / churn_rate) * dataframe['profit_margin']

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(dataframe[["cltv"]])
    dataframe["cltv_c"] = scaler.transform(dataframe[["cltv"]])

    dataframe["cltv_c_segment"] = pd.qcut(dataframe["cltv_c"], 3, labels=["C", "B", "A"])

    dataframe = dataframe[["recency", "frequency", "monetary", "rfm_segment",
                           "cltv_c", "cltv_c_segment"]]

    return dataframe


In [14]:
rfm_cltv = create_cltv_c(rfm)
check_df(rfm_cltv)

##################### Shape #####################
(4338, 6)
##################### Types #####################
recency              int64
frequency            int64
monetary           float64
rfm_segment         object
cltv_c             float64
cltv_c_segment    category
dtype: object
##################### Head #####################
             recency  frequency  monetary  rfm_segment    cltv_c  \
Customer ID                                                        
12346.0          326          1    310.44  hibernating  1.000135   
12347.0            3          7   4310.00    champions  1.025959   
12348.0           76          4   1770.78      at_risk  1.004382   

            cltv_c_segment  
Customer ID                 
12346.0                  C  
12347.0                  A  
12348.0                  A  
##################### NA #####################
recency           0
frequency         0
monetary          0
rfm_segment       0
cltv_c            0
cltv_c_segment    0
dtype: int64

In [15]:
rfm_cltv.head()

,recency,frequency,monetary,rfm_segment,cltv_c,cltv_c_segment
Customer ID,,,,,,
12346.0,326,1,310.44,hibernating,1.000135,C
12347.0,3,7,4310.00,champions,1.025959,A
12348.0,76,4,1770.78,at_risk,1.004382,A
12349.0,19,1,1491.72,promising,1.003110,A
12350.0,311,1,331.46,hibernating,1.000154,C


# Predicted CLTV

In [16]:
def create_cltv_p(dataframe):
    today_date = dt.datetime(2011, 12, 11)

    ## recency kullanıcıya özel dinamik.
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (today_date-date.min()).days,
                                                                lambda date: (today_date - date.min()).days],
                                                'Invoice': lambda num: num.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    rfm.columns = rfm.columns.droplevel(0)

    ## recency_cltv_p
    rfm.columns = ['recency_cltv_p', 'T', 'frequency', 'monetary']

    ## basitleştirilmiş monetary_avg
    rfm["monetary"] = rfm["monetary"] / rfm["frequency"]

    rfm.rename(columns={"monetary": "monetary_avg"}, inplace=True)


    # BGNBD için WEEKLY RECENCY VE WEEKLY T'nin HESAPLANMASI
    ## recency_weekly_cltv_p
    rfm["recency_weekly_cltv_p"] = rfm["recency_cltv_p"] / 7
    rfm["T_weekly"] = rfm["T"] / 7



    # KONTROL
    rfm = rfm[rfm["monetary_avg"] > 0]

    ## recency filtre (daha saglıklı cltvp hesabı için)
    rfm = rfm[(rfm['frequency'] > 1)]

    rfm["frequency"] = rfm["frequency"].astype(int)

    # BGNBD
    bgf = BetaGeoFitter(penalizer_coef=0.01)
    bgf.fit(rfm['frequency'],
            rfm['recency_weekly_cltv_p'],
            rfm['T_weekly'])

    # exp_sales_1_month
    rfm["exp_sales_1_month"] = bgf.predict(4,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])
    # exp_sales_3_month
    rfm["exp_sales_3_month"] = bgf.predict(12,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])

    # expected_average_profit
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(rfm['frequency'], rfm['monetary_avg'])
    rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                             rfm['monetary_avg'])
    # 6 aylık cltv_p
    cltv = ggf.customer_lifetime_value(bgf,
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'],
                                       rfm['monetary_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)

    rfm["cltv_p"] = cltv

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(rfm[["cltv_p"]])
    rfm["cltv_p"] = scaler.transform(rfm[["cltv_p"]])

    # rfm.fillna(0, inplace=True)

    # cltv_p_segment
    rfm["cltv_p_segment"] = pd.qcut(rfm["cltv_p"], 3, labels=["C", "B", "A"])

    ## recency_cltv_p, recency_weekly_cltv_p
    rfm = rfm[["recency_cltv_p", "T", "monetary_avg", "recency_weekly_cltv_p", "T_weekly",
               "exp_sales_1_month", "exp_sales_3_month", "expected_average_profit",
               "cltv_p", "cltv_p_segment"]]


    return rfm



In [17]:
rfm_cltv_p = create_cltv_p(df_prep)
check_df(rfm_cltv_p)

##################### Shape #####################
(2845, 10)
##################### Types #####################
recency_cltv_p                int64
T                             int64
monetary_avg                float64
recency_weekly_cltv_p       float64
T_weekly                    float64
exp_sales_1_month           float64
exp_sales_3_month           float64
expected_average_profit     float64
cltv_p                      float64
cltv_p_segment             category
dtype: object
##################### Head #####################
             recency_cltv_p    T  monetary_avg  recency_weekly_cltv_p  \
Customer ID                                                             
12347.0                 368  368    615.714286              52.571429   
12348.0                 359  359    442.695000              51.285714   
12352.0                 297  297    219.542500              42.428571   

              T_weekly  exp_sales_1_month  exp_sales_3_month  \
Customer ID                         

In [18]:
crm_final = rfm_cltv.merge(rfm_cltv_p, on="Customer ID", how="left")
check_df(crm_final)

##################### Shape #####################
(4338, 16)
##################### Types #####################
recency                       int64
frequency                     int64
monetary                    float64
rfm_segment                  object
cltv_c                      float64
cltv_c_segment             category
recency_cltv_p              float64
T                           float64
monetary_avg                float64
recency_weekly_cltv_p       float64
T_weekly                    float64
exp_sales_1_month           float64
exp_sales_3_month           float64
expected_average_profit     float64
cltv_p                      float64
cltv_p_segment             category
dtype: object
##################### Head #####################
             recency  frequency  monetary  rfm_segment    cltv_c  \
Customer ID                                                        
12346.0          326          1    310.44  hibernating  1.000135   
12347.0            3          7   4310.00    c

In [19]:
crm_final.sort_values(by="monetary_avg", ascending=False).head()

,recency,frequency,monetary,rfm_segment,cltv_c,cltv_c_segment,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,,,,,,
12415.0,25,21,120210.355,loyal_customers,21.193978,A,338.0,338.0,5724.302619,48.285714,48.285714,1.541963,4.625890,5772.177190,50.671759,A
12590.0,212,2,9182.345,hibernating,1.117827,A,212.0,212.0,4591.172500,30.285714,30.285714,0.371947,1.115842,5029.409866,11.422714,A
12435.0,80,2,7829.890,hibernating,1.085674,A,268.0,268.0,3914.945000,38.285714,38.285714,0.311510,0.934530,4288.935706,8.437759,A
14088.0,11,13,50239.210,champions,4.527148,A,323.0,323.0,3864.554615,46.142857,46.142857,1.039334,3.118001,3917.128640,23.708770,A
18102.0,1,60,231584.345,champions,75.947390,A,368.0,368.0,3859.739083,52.571429,52.571429,3.893850,11.681549,3870.996702,85.134766,A


# Veri Tabanına Gönderme

In [21]:
crm_final.head()

,recency,frequency,monetary,rfm_segment,cltv_c,cltv_c_segment,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,,,,,,
12346.0,326,1,310.44,hibernating,1.000135,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,3,7,4310.00,champions,1.025959,A,368.0,368.0,615.714286,52.571429,52.571429,0.556313,1.668939,631.911974,2.941038,A
12348.0,76,4,1770.78,at_risk,1.004382,A,359.0,359.0,442.695000,51.285714,51.285714,0.374986,1.124958,463.745539,1.949217,B
12349.0,19,1,1491.72,promising,1.003110,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12350.0,311,1,331.46,hibernating,1.000154,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
crm_final.index.name = "CustomerID"

In [23]:
crm_final.to_sql(name='crm_final_hasan_kirmizi',
                 con=conn,
                 if_exists='replace',
                 index=True,  # index var o da aşağıdaki
                 index_label="CustomerID")